<a href="https://colab.research.google.com/github/hundredeuk2/Contest/blob/main/understanding_competetion_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"patrickheondeuklee","key":"6720c460e021850cbd1a55c7149b5059"}'}

In [6]:
ls -1ha kaggle.json

kaggle.json


In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [3]:
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle competitions download -c jpx-tokyo-stock-exchange-prediction

 95% 212M/224M [00:00<00:00, 227MB/s]
100% 224M/224M [00:00<00:00, 258MB/s]


In [10]:
!ls

 drive					  'kaggle (1).json'   sample_data
 jpx-tokyo-stock-exchange-prediction.zip   kaggle.json


In [5]:
!unzip /content/jpx-tokyo-stock-exchange-prediction.zip

Archive:  /content/jpx-tokyo-stock-exchange-prediction.zip
  inflating: data_specifications/options_spec.csv  
  inflating: data_specifications/stock_fin_spec.csv  
  inflating: data_specifications/stock_list_spec.csv  
  inflating: data_specifications/stock_price_spec.csv  
  inflating: data_specifications/trades_spec.csv  
  inflating: example_test_files/financials.csv  
  inflating: example_test_files/options.csv  
  inflating: example_test_files/sample_submission.csv  
  inflating: example_test_files/secondary_stock_prices.csv  
  inflating: example_test_files/stock_prices.csv  
  inflating: example_test_files/trades.csv  
  inflating: jpx_tokyo_market_prediction/__init__.py  
  inflating: jpx_tokyo_market_prediction/competition.cpython-37m-x86_64-linux-gnu.so  
  inflating: stock_list.csv          
  inflating: supplemental_files/financials.csv  
  inflating: supplemental_files/options.csv  
  inflating: supplemental_files/secondary_stock_prices.csv  
  inflating: supplemental_fil

In [63]:
import pandas as pd
import seaborn as sns
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import numpy as np

In [29]:
pred_data = pd.read_csv('/content/example_test_files/sample_submission.csv')

In [30]:
pred_data

,Date,SecuritiesCode,Rank
0,2021-12-06,1301,0
1,2021-12-06,1332,1
2,2021-12-06,1333,2
3,2021-12-06,1375,3
4,2021-12-06,1376,4
...,...,...,...
111995,2022-02-28,9990,1995
111996,2022-02-28,9991,1996
111997,2022-02-28,9993,1997
111998,2022-02-28,9994,1998


In [33]:
pred_data[pred_data['Date']=='2021-12-06']

,Date,SecuritiesCode,Rank
0,2021-12-06,1301,0
1,2021-12-06,1332,1
2,2021-12-06,1333,2
3,2021-12-06,1375,3
4,2021-12-06,1376,4
...,...,...,...
1995,2021-12-06,9990,1995
1996,2021-12-06,9991,1996
1997,2021-12-06,9993,1997
1998,2021-12-06,9994,1998


In [34]:
pred_data['Date'].nunique()

56

In [35]:
# submission 데이터가 56일동안 2000개의 종목코드에 달하는 Rank를 예측하는것.

In [31]:
# rank를 예측하는 대회이므로 뭔지 알아보자
# https://www.kaggle.com/code/chumajin/english-ver-easy-to-understand-the-competition/notebook

In [7]:
# 참고 자료에 따르면 Rank를 알기 앞서 Target column의 공식을 이해하고 맞게 구현해야한다.
data = pd.read_csv('/content/train_files/stock_prices.csv')

In [8]:
data.head()

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730
1,20170104_1332,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,1.0,NaN,False,0.012324
2,20170104_1333,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,1.0,NaN,False,0.006154
3,20170104_1376,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1.0,NaN,False,0.011053
4,20170104_1377,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,1.0,NaN,False,0.003026


In [10]:
data.columns

Index(['RowId', 'Date', 'SecuritiesCode', 'Open', 'High', 'Low', 'Close',
       'Volume', 'AdjustmentFactor', 'ExpectedDividend', 'SupervisionFlag',
       'Target'],
      dtype='object')

# Test1 : 한 종목을 가지고 TARGET 얻어내기

In [38]:
t_1 = data[data['SecuritiesCode']==1301].reset_index(drop=True)
t_1

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730
1,20170105_1301,2017-01-05,1301,2743.0,2747.0,2735.0,2738.0,17900,1.0,NaN,False,0.002920
2,20170106_1301,2017-01-06,1301,2734.0,2744.0,2720.0,2740.0,19900,1.0,NaN,False,-0.001092
3,20170110_1301,2017-01-10,1301,2745.0,2754.0,2735.0,2748.0,24200,1.0,NaN,False,-0.005100
4,20170111_1301,2017-01-11,1301,2748.0,2752.0,2737.0,2745.0,9300,1.0,NaN,False,-0.003295
...,...,...,...,...,...,...,...,...,...,...,...,...
1197,20211129_1301,2021-11-29,1301,2975.0,2984.0,2951.0,2951.0,14300,1.0,NaN,False,0.003793
1198,20211130_1301,2021-11-30,1301,2953.0,2997.0,2900.0,2900.0,20500,1.0,NaN,False,0.007558
1199,20211201_1301,2021-12-01,1301,2909.0,2936.0,2909.0,2911.0,10600,1.0,NaN,False,0.016706
1200,20211202_1301,2021-12-02,1301,2949.0,2973.0,2933.0,2933.0,15200,1.0,NaN,False,-0.003689


In [39]:
t_1['C_t+2'] = t_1['Close'].shift(-2)
t_1['C_t+1'] = t_1['Close'].shift(-1)
t_1.head()

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target,C_t+2,C_t+1
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730,2740.0,2738.0
1,20170105_1301,2017-01-05,1301,2743.0,2747.0,2735.0,2738.0,17900,1.0,NaN,False,0.002920,2748.0,2740.0
2,20170106_1301,2017-01-06,1301,2734.0,2744.0,2720.0,2740.0,19900,1.0,NaN,False,-0.001092,2745.0,2748.0
3,20170110_1301,2017-01-10,1301,2745.0,2754.0,2735.0,2748.0,24200,1.0,NaN,False,-0.005100,2731.0,2745.0
4,20170111_1301,2017-01-11,1301,2748.0,2752.0,2737.0,2745.0,9300,1.0,NaN,False,-0.003295,2722.0,2731.0


In [40]:
t_1['rate'] = (t_1['C_t+2'] - t_1['C_t+1']) / t_1['C_t+1']

In [41]:
t_1[['rate','Target']]

,rate,Target
0,0.000730,0.000730
1,0.002920,0.002920
2,-0.001092,-0.001092
3,-0.005100,-0.005100
4,-0.003295,-0.003295
...,...,...
1197,0.003793,0.003793
1198,0.007558,0.007558
1199,0.016706,0.016706
1200,NaN,-0.003689


In [46]:
# t+2 시점으로 산출되었기 때문에 당연히 -2: 는 rate가 없다. 전체가 동일한지 확인해보자
t_1['rate'] == t_1['Target']

0       False
1       False
2       False
3       False
4       False
        ...  
1197    False
1198    False
1199    False
1200    False
1201    False
Length: 1202, dtype: bool

In [43]:
t_1[['rate','Target']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1202 entries, 0 to 1201
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   rate    1198 non-null   float64
 1   Target  1202 non-null   float64
dtypes: float64(2)
memory usage: 18.9 KB


In [49]:
# ??
# 소숫점이 다른가보다
print(t_1['rate'][0],t_1['Target'][0])

0.0007304601899196494 0.0007304601899196


In [57]:
# 소숫점 15자리에서 반올림이 달라서 생긴 일이니 그냥 넘어가도록하자

# Test_2 : Rank란?

In [59]:
t_2 = data[data['Date']=="2017-01-04"].reset_index(drop=True)
t_2

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730
1,20170104_1332,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,1.0,NaN,False,0.012324
2,20170104_1333,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,1.0,NaN,False,0.006154
3,20170104_1376,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1.0,NaN,False,0.011053
4,20170104_1377,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,1.0,NaN,False,0.003026
...,...,...,...,...,...,...,...,...,...,...,...,...
1860,20170104_9990,2017-01-04,9990,1191.0,1233.0,1185.0,1223.0,58600,1.0,NaN,False,-0.022117
1861,20170104_9991,2017-01-04,9991,1117.0,1131.0,1115.0,1129.0,56100,1.0,NaN,False,0.000000
1862,20170104_9993,2017-01-04,9993,1756.0,1784.0,1756.0,1773.0,5300,1.0,NaN,False,0.015042
1863,20170104_9994,2017-01-04,9994,1716.0,1769.0,1716.0,1754.0,27400,1.0,NaN,False,-0.001132


In [60]:
# 우연찮게 발견했지만 하루에 없는 종목코드도 있나보다. 확인해서 결측치 처리 해야할듯!!

In [67]:
# submission과 참고자료에 따르면 2000개의 종목코드 이기 때문에 하루에 0 to 1999 까지 Rank가 있음.
t_2['rank'] = t_2['Target'].rank(ascending=False, method='first') - 1

In [68]:
t_2

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target,rank
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730,898.0
1,20170104_1332,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,1.0,NaN,False,0.012324,325.0
2,20170104_1333,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,1.0,NaN,False,0.006154,567.0
3,20170104_1376,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1.0,NaN,False,0.011053,364.0
4,20170104_1377,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,1.0,NaN,False,0.003026,750.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1860,20170104_9990,2017-01-04,9990,1191.0,1233.0,1185.0,1223.0,58600,1.0,NaN,False,-0.022117,1770.0
1861,20170104_9991,2017-01-04,9991,1117.0,1131.0,1115.0,1129.0,56100,1.0,NaN,False,0.000000,1015.0
1862,20170104_9993,2017-01-04,9993,1756.0,1784.0,1756.0,1773.0,5300,1.0,NaN,False,0.015042,253.0
1863,20170104_9994,2017-01-04,9994,1716.0,1769.0,1716.0,1754.0,27400,1.0,NaN,False,-0.001132,1060.0


In [70]:
t_2 = t_2.sort_values("rank").reset_index(drop=True)

In [71]:
t_2

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target,rank
0,20170104_3825,2017-01-04,3825,146.0,151.0,144.0,144.0,519900,1.0,NaN,False,0.175676,0.0
1,20170104_7600,2017-01-04,7600,834.0,834.0,820.0,827.0,49900,1.0,NaN,False,0.106332,1.0
2,20170104_6425,2017-01-04,6425,3375.0,3455.0,3350.0,3400.0,262500,1.0,NaN,False,0.104668,2.0
3,20170104_3547,2017-01-04,3547,5570.0,5590.0,5480.0,5530.0,9800,1.0,NaN,False,0.098553,3.0
4,20170104_9233,2017-01-04,9233,482.0,492.0,479.0,492.0,5000,1.0,NaN,False,0.094378,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1860,20170104_9974,2017-01-04,9974,4515.0,4610.0,4500.0,4600.0,26400,1.0,NaN,False,-0.059276,1860.0
1861,20170104_6875,2017-01-04,6875,2587.0,2611.0,2562.0,2591.0,192200,1.0,NaN,False,-0.062112,1861.0
1862,20170104_9983,2017-01-04,9983,42480.0,43330.0,42450.0,43290.0,648100,1.0,NaN,False,-0.067387,1862.0
1863,20170104_9474,2017-01-04,9474,2149.0,2156.0,2072.0,2119.0,325800,1.0,NaN,False,-0.069874,1863.0


In [72]:
# 따라서 Target을 알게 되면 Rank를 알 수 있다. 